## Finding missing geocodes in a voter file using Opencage for GA voter file.

In [1]:
#Load libraries/packages
library(tidyverse)
library(readr)
library(stringr)
library(base)
install.packages("opencage")
require(opencage)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Loading required package: opencage



In [5]:
#Set path for voter file data
path <- setwd("/srv/shared/BISG_datasets")

In [6]:
#Opencage key
opencage_key <- "faed2ce713894d20a5758e6b1ecdd62a"

In [7]:
#Load the voter file
voter_file <-read_csv(paste0(path, "/", "georgia_100k_sample.csv"))

Parsed with column specification:
cols(
  .default = col_character(),
  COUNTY_CODE = col_double(),
  REGISTRATION_NUMBER = col_double(),
  RESIDENCE_HOUSE_NUMBER = col_double(),
  RESIDENCE_ZIPCODE = col_double(),
  BIRTHDATE = col_double(),
  REGISTRATION_DATE = col_double(),
  DATE_ADDED = col_double(),
  DATE_CHANGED = col_double(),
  DISTRICT_COMBO = col_double(),
  LAST_CONTACT_DATE = col_double(),
  MAIL_HOUSE_NBR = col_double(),
  MAIL_ZIPCODE = col_double(),
  MAIL_ADDRESS_3 = col_logical(),
  MAIL_COUNTRY = col_logical(),
  lat = col_double(),
  lon = col_double()
)

See spec(...) for full column specifications.

Warning message:
“248 parsing failures.
 row                    col               expected actual                                                file
1573 RESIDENCE_HOUSE_NUMBER no trailing characters      B '/srv/shared/BISG_datasets/georgia_100k_sample.csv'
1788 RESIDENCE_HOUSE_NUMBER no trailing characters      B '/srv/shared/BISG_datasets/georgia_100k_sample.csv'

In [8]:
# Subset a dataframe for voters with no longitude or latitude.
miss_latlon<-which(is.na(voter_file$lat) & is.na(voter_file$lon))

In [9]:
miss_latlon_df <- voter_file[miss_latlon,]

In [10]:
num_miss <- length(miss_latlon)
total_voters<-nrow(voter_file)-num_miss

In [11]:
total_geocoded <- round((total_voters-num_miss)/total_voters*100, 2)
message(paste(total_geocoded, "% of your voter file is now geocoded.", sep=" "))

99.95 % of your voter file is now geocoded.



In [14]:
if(num_miss>0){       
            message(paste("There are", num_miss, "voters missing latitude and longitude values in your voter geocoded voter file.\nWe will use Opencage to attempt to find them now.", sep=" "))
    
            #Let's take out the 47 people that missing geocodes and put in a dataframe
            miss_latlon_df$state <- str_sub(miss_latlon_df$full_addy,-2,-1)

            #Make final address for Opencage to identify

            miss_latlon_df$streetnum_name<- paste(miss_latlon_df$RESIDENCE_HOUSE_NUMBER, 
                                                      miss_latlon_df$RESIDENCE_STREET_NAME, sep=" ")

            miss_latlon_df$final_address<- paste(miss_latlon_df$streetnum_name,
                                                    miss_latlon_df$RESIDENCE_CITY,
                                                    miss_latlon_df$state,
                                                    miss_latlon_df$RESIDENCE_ZIPCODE, sep = ", ")
            
                      
            #Run these addresses through Opencage to identify latitude and longitude
            #Use opencage to find lat and lon for voters without geocodes and put back into ram dataframe.
                    suppressWarnings(
                        for(m in 1:nrow(miss_latlon_df)){
                        opencage_latlon<-opencage_forward(placename = miss_latlon_df$final_address[m], country="US", key=opencage_key)
                        miss_latlon_df$lon[m] <-opencage_latlon$results$geometry.lng
                        miss_latlon_df$lat[m] <-opencage_latlon$results$geometry.lat
                      }
                        )
            #Join the new_lon_lat_df dataframe and the missing_geo_city_zip dataframe.
            voter_file$lon[match(miss_latlon_df$REGISTRATION_NUMBER, voter_file$REGISTRATION_NUMBER)] <- miss_latlon_df$lon
            voter_file$lat[match(miss_latlon_df$REGISTRATION_NUMBER, voter_file$REGISTRATION_NUMBER)] <- miss_latlon_df$lat

            #Check for missing geocodes again
            miss_latlon2<-which(is.na(voter_file$lat) & is.na(voter_file$lon==0))
            num_miss2<-length(miss_latlon2)
            total_geocoded2<- round((total_voters-num_miss2)/total_voters*100, 2)
            
            message(paste(total_geocoded2, "% of your voter file is now geocoded using the Opencage API.", sep=" "))
            new_voter_file <- voter_file
        
    return(new_voter_file)  
} else {
    message(paste("There are no missing geocodes (i.e. latitudes and longitudes) in the voter file.", sep=" "))
}
    
    
 

There are 47 voters missing latitude and longitude values in your voter geocoded voter file.
We will use Opencage to attempt to find them now.

100 % of your voter file is now geocoded using the Opencage API.



full_addy,COUNTY_CODE,REGISTRATION_NUMBER,VOTER_STATUS,LAST_NAME,FIRST_NAME,MIDDLE_MAIDEN_NAME,NAME_SUFFIX,RESIDENCE_HOUSE_NUMBER,RESIDENCE_STREET_NAME,⋯,MAIL_STREET_NAME,MAIL_APT_UNIT_NBR,MAIL_CITY,MAIL_STATE,MAIL_ZIPCODE,MAIL_ADDRESS_2,MAIL_ADDRESS_3,MAIL_COUNTRY,lat,lon
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<lgl>,<dbl>,<dbl>
"MISSING ADDRESS , , GA",60,10356431,A,BOONE,DONTE,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,32.16562,-82.90008
"MISSING ADDRESS , , GA",25,12264902,A,MYERS,JOSH,DAVID,NA,8112,WHITE BLUFF RD,⋯,WHITE BLUFF RD,NA,SAVANNAH,GA,314063406,NA,NA,NA,32.16562,-82.90008
"MISSING ADDRESS , , GA",8,7764707,A,MCLENDON,ANTHONY,LEON,NA,269,MARKET PLACE BLVD,⋯,MARKET PLACE BLVD,#186,CARTERSVILLE,GA,301212235,NA,NA,NA,32.16562,-82.90008
"MISSING ADDRESS , , GA",48,11584512,A,MACK,NADIE',GENENE,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,32.16562,-82.90008
"MISSING ADDRESS , , GA",137,2648120,A,LOUIS,ROBERT,LEE,NA,205,JOHN HOWARD WAY,⋯,JOHN HOWARD WAY,UNIT 599,TIFTON,GA,317936032,NA,NA,NA,32.16562,-82.90008
"MISSING ADDRESS , , GA",25,8041842,A,MYRICK,MYRA,REBECCA ANN,NA,250,MARTIN LUTHER KING BLVD,⋯,MARTIN LUTHER KING BLVD,NA,SAVANNAH,GA,314014247,NA,NA,NA,32.16562,-82.90008
"MISSING ADDRESS , , GA",47,11080050,A,THOMAS,JOHN,T,NA,NA,MISSING ADDRESS,⋯,NA,NA,NA,NA,NA,NA,NA,NA,32.16562,-82.90008
"SAPELO IS 31327, SAPELO ISLAND, GA",98,2315982,A,GOEKJIAN,VIRGINIA,H,NA,NA,SAPELO IS,⋯,PO BOX 95,NA,SAPELO ISLAND,GA,313270095,NA,NA,NA,31.41519,-81.25739
"1 14TH AVE NW 39828, CAIRO, GA",65,608907,A,HAMMETT,TINA,P,NA,1,14TH AVE NW,⋯,NA,NA,NA,NA,NA,NA,NA,NA,30.89177,-84.20798
